# Seaborn: Concentration vs. Extraction Time

In [1]:
import csv
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
%matplotlib inline
sns.set_style("white") #Sets the background for the plot 

#sns.set_style("white")
#sns.set_style("whitegrid")
#sns.set_style("dark")
#sns.set_style("darkgrid")

In [2]:
ps = pd.read_csv("patch_seq_log_mouse.csv")

In [3]:
initial =["Date", "Post patch?", "Post patch pipette R", "Time spent extracting cytosol", "BA conc. >400 (pg/uL)", 
          "PCR cycles", "SM_QC_PF", "Bad dates"]
ps = ps[initial]

#Changing Date Format
ps["Date"] = pd.to_datetime(ps["Date"], format = "%y%m%d")

#Changing String to Numerical
ps["Time spent extracting cytosol"] = pd.to_numeric(ps["Time spent extracting cytosol"].str.replace(' ',''), errors ='coerce')
ps["BA conc. >400 (pg/uL)"] = pd.to_numeric(ps["BA conc. >400 (pg/uL)"].str.replace(' ',''), errors ='coerce')

#Categories
ps["Post patch?"] = ps["Post patch?"].astype("category")
ps["PCR cycles"] = ps["PCR cycles"].astype("category")
ps["SM_QC_PF"] = ps["SM_QC_PF"].astype("category")
ps["Bad dates"] = ps["Bad dates"].astype("category")

#Filtering Columns
ps = ps[ps["PCR cycles"] == 21]      #Shows 21 pcr cycles
ps = ps[ps["SM_QC_PF"] != "cnt"]     #Shows everything except cnt
ps = ps[ps["Bad dates"] != "x"]      #Shows everthing except x
ps = ps[ps["Time spent extracting cytosol"] <= 10]
ps = ps[ps["BA conc. >400 (pg/uL)"] < 8000]

#Dropping blank columns
ps = ps.drop("Bad dates", 1)
ps.dropna(subset = ["Post patch?", "Time spent extracting cytosol", "BA conc. >400 (pg/uL)", "SM_QC_PF"], inplace = True)
ps.head()

TypeError: 'int' object is unsliceable

## Definitions

In [ ]:
def nucleated(x):
    nucleus = x[(x["Post patch?"] == "Nucleated") | (((x["Post patch?"] == "nucleus_visible") | (x["Post patch?"] == "nucleus_present")) & (x["Post patch pipette R"] >= 500))]
    return nucleus

def partial_nucleated(y):
    partial = y[(y["Post patch?"] == 'Partial-Nucleus') | (((y['Post patch?'] == 'nucleus_present') | (y['Post patch?'] == 'nucleus_visible')) & (y["Post patch pipette R"] <= 499))]
    return partial

def outside_out(z):
    outside = z[(z["Post patch?"] == 'Outside-Out') | (((z['Post patch?'] == 'nucleus_absent') | (z['Post patch?'] == 'no_nuclues_visible')) & (z['Post patch pipette R'] >= 500))]
    return outside

def no_seal(w): 
    no = w[(w["Post patch?"] == 'No-Seal') | (((w['Post patch?'] == 'nucleus_absent') | (w['Post patch?'] == 'no_nucleus_visible')) & (w['Post patch pipette R'] <= 499))]
    return no
    
def entire_cell(v):
    entire = v[(v["Post patch?"] == 'Entire-Cell') | (v['Post patch?'] == 'entire_cell')]
    return entire

## Setting Variables

In [ ]:
oo = outside_out(ps)
oo['post patch class'] = 'Outside out'
nu = nucleated(ps)
nu['post patch class'] = 'Nucleated'
ns = no_seal(ps)
ns['post patch class'] = 'No seal'
pn = partial_nucleated(ps)
pn['post patch class'] = 'Partial nucleated'

#Not including entire cell with plot
#ec = entire_cell(ps)
#ec['post patch class'] = 'Entire cell'

frames = (nu, pn, oo, ns)
ps = pd.concat(frames)
ps.head()

## Plotting Trend Line: Concentration vs Extraction Time (Seaborn)

In [ ]:
lm = sns.lmplot(x="Time spent extracting cytosol", y="BA conc. >400 (pg/uL)", data=ps, hue="post patch class",
                scatter=False, legend=False)

axes = lm.axes #Set array of axes
axes[0,0].set_ylim(0,2000) #Sets axes as center coordinate & y limits

plt.title("Concentration vs. Extraction Time") #Create plot title
lm.set(xlabel="Extraction Time (min)",ylabel="Concentration >400 (pg/uL)") #Sets x and y axis labels 
lm.ax.legend(loc=2) #Creates manual legend with movable location

#loc is determined by quadrants 
#loc1 = quadrant 1
#loc2 = quadrant 2
#loc3 = quadrant 3
#loc4 = quadrant 4

#sns.despine() #Gets rid of the top and right spine on plot
plt.tight_layout() #Moves axes into place if out of frame
plt.savefig("conc_vs_extraction_seaborn.png",dpi=300) #Saves plot as .png